In [118]:
import pickle
import numpy as np
from joblib import load
from sklearn.metrics import f1_score

In [119]:
# Lista com todas as classes.
probs_cols = [
    'mirai',
    'unk_bruteforcer',
    'unk_spammer',
    'shadowserver',
    'driftnet',
    'internetcensus',
    'censys',
    'rapid7',
    'onyphe',
    'netsystems',
    'shodan',
    'unk_exploiter',
    'securitytrails',
    'intrinsec',
    'unknown']

probs_cols.sort()

In [120]:
probs_cols

['censys',
 'driftnet',
 'internetcensus',
 'intrinsec',
 'mirai',
 'netsystems',
 'onyphe',
 'rapid7',
 'securitytrails',
 'shadowserver',
 'shodan',
 'unk_bruteforcer',
 'unk_exploiter',
 'unk_spammer',
 'unknown']

In [121]:
label_to_idx = { l:idx for idx, l in enumerate(probs_cols) }

In [122]:
source_dir = "stacking_data/vanilla/20211221/idarkvec-igcngru_features"

In [123]:
def models_miss(data, label_to_idx, model):

    m1 = data["X_test"][: , :15]
    m2 = data["X_test"][: , 15:]
    st = data["probas"]

    pm1 = m1.argmax(axis=1)
    pm2 = m2.argmax(axis=1)
    
    # Ajustando os índices das predições do stacking.
    # Com o ajuste podemos indexar a lista de classes completas 
    # para extrair as predições do stacking.
    pst = st.argmax(axis=1)
    pst = [ model.classes_[p] for p in pst ]
    pst = np.array([ label_to_idx[p] for p in pst ])
    
    y = np.array([ label_to_idx[l] for l in data["y_test"] ])
    
    print("Macros: (idarkvec, igcngru-features, stacking)", f1_score(y, pm1, average="macro"), f1_score(y, pm2, average="macro"), f1_score(y, pst, average="macro"))
    
    c1 = pm1 != y
    c2 = pm2 != y
    cst = pst == y
    return c1 & c2 & cst

In [124]:
hits = {}
for fold in np.arange(10):
    hits[fold] = {}
    data = np.load(f"{source_dir}/{fold}/data.npz", allow_pickle=True)
    model = load(f"{source_dir}/{fold}/lr.joblib")
    idxs = models_miss(data, label_to_idx, model)
    hits[fold]["X"] = data["X_test"][idxs]
    hits[fold]["y"] = data["y_test"][idxs]
    hits[fold]["idxs"] = np.arange(data["y_test"].shape[0])[idxs]
    hits[fold]["stacking_probas"] = data["probas"]
     

Macros: (idarkvec, igcngru-features, stacking) 0.728209652450947 0.756194851379166 0.9238067030090724
Macros: (idarkvec, igcngru-features, stacking) 0.7260275406040763 0.6838488425914896 0.8551559011169542
Macros: (idarkvec, igcngru-features, stacking) 0.7288893884133655 0.7779355253449233 0.8813236087236395
Macros: (idarkvec, igcngru-features, stacking) 0.7675103786630227 0.7640022929639055 0.8986869232770872
Macros: (idarkvec, igcngru-features, stacking) 0.7130406129129352 0.714118645138592 0.8736030659315048
Macros: (idarkvec, igcngru-features, stacking) 0.7741809135506144 0.7413304254394527 0.9237228382609142
Macros: (idarkvec, igcngru-features, stacking) 0.736892781277134 0.7210164737028057 0.90265486889463
Macros: (idarkvec, igcngru-features, stacking) 0.7529453214048705 0.789764017242117 0.8938442988963645
Macros: (idarkvec, igcngru-features, stacking) 0.784757997780946 0.7038592792325032 0.8331893169348018
Macros: (idarkvec, igcngru-features, stacking) 0.7234680620343931 0.6759

# Teste de sanidade.

In [125]:
hits[0]["idxs"]

array([103, 275, 299, 334, 349, 353, 354, 367, 376, 377, 379, 395, 396,
       402])

In [126]:
hits[0]["y"]

array(['unk_bruteforcer', 'unk_spammer', 'mirai', 'unk_bruteforcer',
       'unk_spammer', 'unk_spammer', 'unk_bruteforcer', 'unk_bruteforcer',
       'unk_spammer', 'mirai', 'unk_bruteforcer', 'unk_bruteforcer',
       'unk_bruteforcer', 'unk_bruteforcer'], dtype=object)

In [164]:
TEST_IDX = 334
LABEL = hits[0]["y"][np.where(hits[0]["idxs"] == TEST_IDX)[0][0]]

### Pegando o primeiro caso (idx 103 - unk_bruteforcer) onde o stacking acertou e os modelos erraram.

In [165]:
pred_model_idarkvec = hits[0]["X"][0][:15].argmax(axis=0)
pred_model_idarkvec, probs_cols[pred_model_idarkvec]

(14, 'unknown')

In [166]:
pred_model_igcngru = hits[0]["X"][0][15:].argmax(axis=0)
pred_model_igcngru, probs_cols[pred_model_igcngru]

(14, 'unknown')

#### Indexando as predições do stacking com o ID 103 sem ajustar o índice

In [167]:
pred_stacking = hits[0]["stacking_probas"][TEST_IDX].argmax()
print("Predição do stacking sem ajustar o índice para as 15 classes: ", pred_stacking, probs_cols[pred_stacking], LABEL)
print("Pode ver que está errada.")


Predição do stacking sem ajustar o índice para as 15 classes:  10 shodan unk_bruteforcer
Pode ver que está errada.


#### Indexando as predições do stacking com o ID 103 usando as classes que o modelo encontrou durante o treinamento.

In [168]:
pred_stacking_id = hits[0]["stacking_probas"][TEST_IDX].argmax()
model = load("stacking_data/vanilla/20211221/idarkvec-igcngru_features/0/lr.joblib")
pred_stacking = model.classes_.tolist()[pred_stacking_id]
print("Predição do stacking indexando as classes salvas no modelo: ", pred_stacking_id, pred_stacking, LABEL)
print("Pode ver que está correto.")

Predição do stacking indexando as classes salvas no modelo:  10 unk_bruteforcer unk_bruteforcer
Pode ver que está correto.


#### Indexando as predições do stacking com o ID 103 convertendo as predições do stacking para os IDs englobando todas as classes.

In [169]:
pred_stacking_id = hits[0]["stacking_probas"][TEST_IDX].argmax()

model = load("stacking_data/vanilla/20211221/idarkvec-igcngru_features/0/lr.joblib")

pred_stacking = model.classes_.tolist()[pred_stacking_id]

pred_stacking_id_adjusted = label_to_idx[pred_stacking]

print("Predição do stacking convertendo o ID da sua predição para as 15 classes (igual os demais modelos): ", pred_stacking_id_adjusted, probs_cols[pred_stacking_id_adjusted], LABEL)
print("Pode ver que está correto usando a lista de classes probs_cols - o mesmo para os modelos base do stacking.")

Predição do stacking convertendo o ID da sua predição para as 15 classes (igual os demais modelos):  11 unk_bruteforcer unk_bruteforcer
Pode ver que está correto usando a lista de classes probs_cols - o mesmo para os modelos base do stacking.


In [170]:
with open("filtered_preds.pkl", 'wb') as fd:
    pickle.dump(hits, fd)